<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-12 14:27:05
-------------------
qualified stocks: 91
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  3.50 L
Current:  1.35 C
-------------------
Today PnL: -1.32 L (-0.97%)
Current PnL: -15.47 L (-10.78%)
CY Booked + Current PnL: -9.10 L (-6.34%)
-------------------
Total profit:  3.16 L
Total loss:  -18.63 L
-------------------
Total Booked + Current PnL: 17.88 L (15.35%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.74%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 71.37 L (53.06%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-2.27,H-LC,85.71,99374.0,14187.0,1878.0,1.83,16.65,1.89,18.85,35.0,7.55,0.76,32.31,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.0,2.51,M-LC,53.85,102716.0,24354.0,9676.0,-0.67,31.08,9.42,43.43,77.0,2.52,0.78,37.38,XY24,NTT,MISC
60,RELIANCE,1291.83,1526.0,5.36,X-LC,31.87,156655.0,15846.0,9681.0,-0.81,11.25,6.18,18.13,17.0,1.64,1.20,23.69,XY25,NTT,REFINERIES
82,UJJIVANSFB,52.77,53.0,51.69,M-SC,81.32,131274.0,-11205.0,11828.0,-0.80,-7.86,9.01,0.44,242.0,-0.95,1.00,54.94,OX40N,NTT,BANKS
76,TATAELXSI,7332.28,7332.0,-13.80,X-MC,56.04,90594.0,-12058.0,12058.0,-2.32,-11.75,13.31,-0.00,57.0,-1.00,0.69,37.71,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,130.64,228.00,97.69,M-MC,70.33,224928.0,-34.0,167684.0,-1.90,-0.02,74.55,74.53,184.0,-0.00,1.72,30.59,XY24,BTT,STEEL
13,BERGEPAINT,561.33,680.00,-16.51,H-MC,47.25,228764.0,1425.0,46645.0,-0.41,0.63,20.39,21.14,117.0,0.03,1.75,28.08,XY24,NTT,PAINTS
56,PGHH,13388.00,18106.42,-20.27,X-MC,8.79,201225.0,405.0,70368.0,0.00,0.20,34.97,35.24,53.0,0.01,1.54,4.89,X40,ATH,FMCG
5,APOLLOHOSP,6904.43,8285.00,-15.34,H-LC,35.16,104550.0,984.0,19729.0,0.61,0.95,18.87,20.00,32.0,0.05,0.80,15.16,X40N,BTT,HEALTHCARE
19,CIPLA,1495.00,1795.00,-17.66,H-LC,19.78,100400.0,235.0,19869.0,-1.52,0.23,19.79,20.07,29.0,0.01,0.77,8.22,X40N,BTT,PHARMA


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TANLA,917.30,1963.11,-26.57,H-SC,97.80,179946.0,-59469.0,332432.0,10.73,-24.84,184.74,114.01,128.0,-0.18,1.37,65.04,AR,ATH,IT
62,ROUTE,1558.40,2249.26,-33.31,M-SC,86.81,71930.0,-30924.0,76519.0,6.12,-30.07,106.38,44.33,188.0,-0.40,0.55,19.45,SR,ATH,IT
7,ASIANTILES,75.41,137.00,7238.89,M-SC,98.90,80549.0,-13261.0,89877.0,4.22,-14.14,111.58,81.67,238.0,-0.15,0.61,55.99,XR,NTT,CERAMICS
85,VALIANTORG,512.64,838.00,-402.56,H-SC,100.00,128700.0,-4586.0,89176.0,2.35,-3.44,69.29,63.47,142.0,-0.05,0.98,115.73,XR,NTT,CHEMICALS
50,LICI,827.06,983.00,-2.27,H-LC,85.71,99374.0,14187.0,1878.0,1.83,16.65,1.89,18.85,35.0,7.55,0.76,32.31,XY25,NTT,INSURANCE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69120.0,-44429.0,84478.0,-3.56,-39.13,122.22,35.27,260.0,-0.53,0.53,69.17,XR,NTT,HOTELS
17,CAMS,3643.00,5250.99,2.00,H-SC,74.73,114604.0,12600.0,32421.0,-3.21,12.35,28.29,44.14,125.0,0.39,0.87,30.60,X40N,ATH,MISC
57,QUESS,711.66,986.00,NaN,NaN,1.10,67091.0,-88763.0,148841.0,-3.05,-56.95,221.85,38.55,191.0,-0.60,0.51,2.73,XY24,NTT,MISC
9,AWL,325.39,485.00,-68.61,H-MC,41.76,220894.0,-47553.0,179233.0,-2.99,-17.71,81.14,49.05,98.0,-0.27,1.69,13.57,XY24,NTT,FMCG
27,GREENPANEL,375.16,537.00,198.45,M-SC,89.01,133273.0,-49805.0,128782.0,-2.86,-27.20,96.63,43.14,212.0,-0.39,1.02,25.36,XY24,NTT,MISC


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UJJIVANSFB,52.77,53.0,51.69,M-SC,81.32,131274.0,-11205.0,11828.0,-0.80,-7.86,9.01,0.44,242.0,-0.95,1.00,54.94,OX40N,NTT,BANKS
74,SYMPHONY,1306.42,1306.0,-28.04,M-SC,3.30,153283.0,-17858.0,17796.0,-1.66,-10.43,11.61,-0.03,190.0,-1.00,1.17,10.17,OX40N,NTT,DURABLES
76,TATAELXSI,7332.28,7332.0,-13.80,X-MC,56.04,90594.0,-12058.0,12058.0,-2.32,-11.75,13.31,-0.00,57.0,-1.00,0.69,37.71,OX40N,NTT,IT
87,VIPIND,488.80,489.0,-950.28,H-SC,93.41,81003.0,-14313.0,14354.0,0.44,-15.02,17.72,0.04,153.0,-1.00,0.62,59.77,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.67,H-SC,9.89,224640.0,-45027.0,81365.0,-0.48,-16.70,36.22,13.47,144.0,-0.55,1.71,12.44,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-10.27,X-LC,32.97,287868.0,26267.0,119350.0,-1.40,10.04,41.46,55.67,1.0,0.22,2.20,16.76,X40,BTT,IT
78,TCS,3794.03,4998.00,-18.09,X-LC,7.69,267072.0,-28862.0,122773.0,-1.38,-9.75,45.97,31.73,2.0,-0.24,2.04,6.90,X40,BTT,IT
34,HINDUNILVR,2413.81,2723.00,-16.79,X-LC,15.38,261184.0,-9163.0,43801.0,-1.87,-3.39,16.77,12.81,5.0,-0.21,1.99,8.10,XY25,NTT,FMCG
14,BRITANNIA,4983.33,6446.05,11.78,X-LC,21.98,256404.0,27171.0,40102.0,-1.35,11.85,15.64,29.35,7.0,0.68,1.96,21.83,XY25,ATH,FMCG
55,NESTLEIND,2268.60,2755.00,-12.59,X-LC,24.18,277711.0,12285.0,44628.0,-1.88,4.63,16.07,21.44,10.0,0.28,2.12,11.11,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UNIONBANK,123.87,163.0,-1.84,M-LC,87.91,171391.0,30551.0,13934.0,-1.36,21.69,8.13,31.59,86.0,2.19,1.31,48.88,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.51,M-LC,53.85,102716.0,24354.0,9676.0,-0.67,31.08,9.42,43.43,77.0,2.52,0.78,37.38,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.00,2.51,M-LC,53.85,102716.0,24354.0,9676.0,-0.67,31.08,9.42,43.43,77.0,2.52,0.78,37.38,XY24,NTT,MISC
17,CAMS,3643.00,5250.99,2.00,H-SC,74.73,114604.0,12600.0,32421.0,-3.21,12.35,28.29,44.14,125.0,0.39,0.87,30.60,X40N,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.00,H-MC,64.84,152796.0,1883.0,58567.0,-1.01,1.25,38.33,40.06,100.0,0.03,1.17,24.98,AR,ATH,AUTO
83,UNIONBANK,123.87,163.00,-1.84,M-LC,87.91,171391.0,30551.0,13934.0,-1.36,21.69,8.13,31.59,86.0,2.19,1.31,48.88,XY24,NTT,BANKS
90,WIPRO,243.46,420.00,-6.08,M-LC,36.26,160729.0,9784.0,99668.0,0.12,6.48,62.01,72.51,68.0,0.10,1.23,10.88,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,WHIRLPOOL,1167.49,2270.00,-32.27,M-SC,51.65,197392.0,25771.0,136299.0,-0.50,15.02,69.05,94.43,194.0,0.19,1.51,45.69,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-6.08,M-LC,36.26,160729.0,9784.0,99668.0,0.12,6.48,62.01,72.51,68.0,0.10,1.23,10.88,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-28.22,M-MC,75.82,189020.0,9212.0,103205.0,-0.85,5.12,54.60,62.52,170.0,0.09,1.44,36.03,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.23,H-SC,65.93,132187.0,8851.0,125974.0,-0.16,7.18,95.30,109.32,122.0,0.07,1.01,30.59,AR,ATH,MISC
32,HEROMOTOCO,4311.81,6039.03,0.00,H-MC,64.84,152796.0,1883.0,58567.0,-1.01,1.25,38.33,40.06,100.0,0.03,1.17,24.98,AR,ATH,AUTO


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-53.72,L-SC,91.21,69120.0,-44429.0,84478.0,-3.56,-39.13,122.22,35.27,260.0,-0.53,0.53,69.17,XR,NTT,HOTELS
52,MASFIN,326.60,399.50,-22.31,H-SC,63.74,88050.0,-9930.0,31804.0,-2.62,-10.13,36.12,22.32,141.0,-0.31,0.67,27.70,XR,ATH,FINANCE
50,LICI,827.06,983.00,-2.27,H-LC,85.71,99374.0,14187.0,1878.0,1.83,16.65,1.89,18.85,35.0,7.55,0.76,32.31,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,2.51,M-LC,53.85,102716.0,24354.0,9676.0,-0.67,31.08,9.42,43.43,77.0,2.52,0.78,37.38,XY24,NTT,MISC
35,HINDZINC,514.80,744.74,37.93,H-LC,90.11,103130.0,170.0,45821.0,-0.94,0.17,44.43,44.67,25.0,0.00,0.79,33.09,X5K,ATH,METALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VALIANTORG,512.64,838.00,-402.56,H-SC,100.0,128700.0,-4586.0,89176.0,2.35,-3.44,69.29,63.47,142.0,-0.05,0.98,115.73,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,7238.89,M-SC,98.9,80549.0,-13261.0,89877.0,4.22,-14.14,111.58,81.67,238.0,-0.15,0.61,55.99,XR,NTT,CERAMICS
75,TANLA,917.30,1963.11,-26.57,H-SC,97.8,179946.0,-59469.0,332432.0,10.73,-24.84,184.74,114.01,128.0,-0.18,1.37,65.04,AR,ATH,IT
46,JPPOWER,18.73,26.20,-33.95,L-SC,96.7,134572.0,-7270.0,63841.0,-2.47,-5.13,47.44,39.88,258.0,-0.11,1.03,36.38,XY24,NTT,POWER
79,TITAGARH,1097.23,1548.00,-9.25,H-SC,95.6,159408.0,-32607.0,111490.0,-2.72,-16.98,69.94,41.08,151.0,-0.29,1.22,34.92,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.71
1,25,43.19
2,50,72.02


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.99
XY25     14.29
X40      12.52
XR       11.06
OX40N     8.53
X40N      8.48
AR        8.14
X5K       2.34
X200      1.49
SR        1.20
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.82
X-LC    15.51
M-SC    14.41
H-LC    11.09
H-MC     8.98
M-LC     8.57
X-MC     7.70
M-MC     5.80
X-SC     2.81
L-SC     2.50
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.63,-2.26,36.48
IT,11.41,-11.17,67.64
BANKS,8.57,-8.34,52.01
MISC,6.61,-24.86,73.69
FINANCE,6.37,-20.41,66.74
PAINTS,5.68,-17.31,40.44
ELECTRICAL,5.15,-4.04,49.17
HEALTHCARE,4.75,-3.51,33.75
AUTO,4.75,-8.71,56.52


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2741101.0,25
XR,1111016.0,13
AR,1003286.0,9
X40,681693.0,9
XY25,518665.0,10
X40N,418737.0,8
OX40N,314755.0,11
SR,187335.0,2
X5K,95639.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1898269.0,19
M-SC,1442002.0,18
X-LC,633057.0,9
M-MC,583971.0,5
H-MC,552086.0,7
H-LC,503774.0,10
X-MC,409378.0,6
M-LC,321963.0,6
L-SC,271203.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,901082.0,6
M-SC,XY24,577770.0,5
H-SC,AR,512533.0,3
M-MC,XY24,393626.0,3
M-SC,XR,341774.0,4
X-LC,X40,321628.0,3
H-MC,XY24,225878.0,2
M-LC,XY24,206895.0,4
H-SC,XR,193657.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
